In [ ]:
import pandas
import torch
from torch.utils.data import random_split
from shared_code.fine_tuning.tensor_encoding.tensor_encoding import TokenizerAdapter
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import TrainingArguments, Trainer

In [ ]:
model_name = "yuri-bot"

parent_directory = "/content/drive/MyDrive/RawData"

model_output_dir = f"{parent_directory}/{model_name}"

tokenizer_path = f"{model_output_dir}"

training_data_path = f"/content/drive/MyDrive/RawData/training.csv"

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium',
										  bos_token='<|startoftext|>',
										  eos_token='<|endoftext|>',
										  pad_token='<|pad|>')

tokenizer.save_pretrained(model_output_dir)

model = GPT2LMHeadModel.from_pretrained('gpt2-medium').cuda()

tokenizer_adapter = TokenizerAdapter(tokenizer)

In [ ]:
df = pandas.read_csv(training_data_path)

conversations = list(df['TrainingString'])

valid_lines = []
for conversation in conversations:
	if tokenizer_adapter.token_length_appropriate(conversation):
		valid_lines.append(conversation)

In [ ]:
from shared_code.fine_tuning.datasets.reddit_dataset import RedditDataset

generator = torch.Generator()

generator.manual_seed(0)

print(f":: Total Number Of Samples {len(valid_lines)}")

max_length = max([len(tokenizer.encode(prompt)) for prompt in valid_lines])

model.resize_token_embeddings(len(tokenizer))

print(f":: Max Length Of Sample {max_length}")

dataset = RedditDataset(valid_lines, tokenizer, max_length=max_length)

train_size = int(0.9 * len(dataset))

train_dataset, eval_dataset = random_split(dataset, [train_size, len(dataset) - train_size], generator=generator)

In [ ]:
training_args = TrainingArguments(output_dir=model_output_dir)
training_args.num_train_epochs = 5
training_args.logging_steps = 100
training_args.save_steps = 1000
training_args.weight_decay = 0.05
training_args.logging_dir = './logs'
training_args.fp16 = True
training_args.auto_find_batch_size = True
training_args.gradient_accumulation_steps = 50
training_args.learning_rate = 1e-4

In [ ]:
trainer: Trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset,
						   data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
													   'attention_mask': torch.stack([f[1] for f in data]),
													   'labels': torch.stack([f[0] for f in data])
													   })

In [ ]:
trainer.train()

trainer.save_model()

In [ ]:
expecte_prompt = "I love these bizarro eras of weirdness in AI development after it becomes possible to do but before it's perfected. Image synthesis itself was in the same place for several years back in the mid to late 2010s."

prompt = "<|soss r/dalle2|><|sot|>Detailed scientific diagram depicting the anatomy of a tomato, full colour, realistic<|sost|>https://i.imgur.com/7adBOXn.jpg<|sor u/AsterJ|>It's going to be sad day when it learns to properly spell.  I feel like this era is a fleeting moment in AI history.  We must cherish it.<|eor|><|sor"

generated = tokenizer(f"<|startoftext|> {prompt}", return_tensors="pt")

sample_outputs = model.generate(inputs=generated.input_ids.cuda(),
								attention_mask=generated['attention_mask'].cuda(),
                                do_sample=True,
                                top_k=40,
								max_length=1024,
                                top_p=0.8,
                                temperature=0.8,
                                num_return_sequences=10,
								repetition_penalty=1.08,
                                stop_token='<|endoftext|>')

for i, sample_output in enumerate(sample_outputs):
    result = tokenizer.decode(sample_output, skip_special_tokens=True)
    print("{}: {}".format(i, result.replace(prompt, "")))